In [1]:
from google.colab import drive
 
# Accessing My Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


# Prequisite

In [2]:
!pip install hazm
!pip install tqdm

from tqdm import tqdm
import numpy as np
import cv2 as cv
import pandas as pd
from hazm import *
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
import math
import re
import csv 

     |████████████████████████████████| 316 kB 15.2 MB/s 
     |████████████████████████████████| 1.4 MB 58.8 MB/s 
     |████████████████████████████████| 233 kB 60.1 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=f0208351fa5c2addb236def410e5a1a57e445a3b0ee63928e0c0a21e1cc7e967
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154428 sha256=e9be72c2951a5d38b68d130bff5bbc8427c086ef6285b559560b7a3993ec57dc
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [3]:
! pip install transformers
from transformers import BertConfig, BertTokenizer, AutoModel
from transformers import BertModel
from transformers import pipeline
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import hazm

import torch
import torch.nn as nn
import torch.nn.functional as F

import os
import re

     |████████████████████████████████| 3.5 MB 11.6 MB/s 
     |████████████████████████████████| 895 kB 45.8 MB/s 
     |████████████████████████████████| 67 kB 6.8 MB/s 
     |████████████████████████████████| 6.8 MB 46.9 MB/s 
     |████████████████████████████████| 596 kB 64.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Helper function

In [4]:
def read_csv(path):
  if "analogy_slang.csv" in path:
    df = pd.read_csv(path, engine='python',encoding='utf-8', error_bad_lines=False)
    df.columns = ["cat", "first", "second", "third","fourth"]
  elif "SAT_ANALOGY" in path:
    df = pd.read_csv(path, engine='python',encoding='utf-8', error_bad_lines=False)
  else:
    df = pd.read_csv(path, engine='python',encoding='utf-8', error_bad_lines=False,  names=["cat", "first", "second", "third","fourth"])
  print("The size of the analogy test set is : {}".format(len(df)))
  print(df.head())
  return df

def save_csv(path, data):
  data.to_csv(path)

def txt_to_csv(path, saved_path):
  data = { 'cat' : [],
          'first': [], 
          'second': [],
          'third' : [],
          'fourth' : [],
          }
  with open(path) as f:
    lines = f.readlines()
    print(len(lines))
    cat = ': Adjective-Adverb'
    for line in lines:
        line = re.sub('\n','', line)
        words = line.split('\t')
        if len(words) == 4 :
          data['cat'].append(cat)
          data['first'].append(words[0])
          data['second'].append(words[1])
          data['third'].append(words[2])
          data['fourth'].append(words[3])
        else:
          cat = words[0]
    data = pd.DataFrame(data)
    print(data.head())
    save_csv(saved_path,data)
    return data

def text_to_csv_2(url, url_2):
  with open(url, "r") as in_text:
      in_reader = csv.reader(in_text, delimiter = '\t')
      with open(url_2, "w") as out_csv:
          out_writer = csv.writer(out_csv)
          for row in in_reader:
              out_writer.writerow(row)
  df = pd.read_csv(url_2, engine='python',encoding='utf-8', error_bad_lines=False)
  return df
      
def process_sat_analogy(path, saved_path):
  data = { 'cat' : [],
          'first': [], 
          'second': [],
          'third' : [],
          'fourth' : [],
          }
  df = read_csv(path)
  print("Number of unique description in sat test : {}".format(len(np.unique(df["Desc"]))))
  for desc in np.unique(df["Desc"]):
    # print(desc)
    df_new = df[df["Desc"] == desc]
    # print(len(df_new))
    w1 = np.asarray(df_new["W1"])
    w2 = np.asarray(df_new["W2"])
    for i in range(0, len(df_new)-1):
      for j in range(i+1, len(df_new)):
          data['cat'].append(desc)
          data['first'].append(w1[i])
          data['second'].append(w2[i])
          data['third'].append(w1[j])
          data['fourth'].append(w2[j])
  data = pd.DataFrame(data)
  # print(data.head())
  save_csv(saved_path,data)
  return df , data


In [5]:
def pre_proc(item):
  item = re.sub("\'",'',item)
  item = re.sub("[",'',item)
  item = re.sub("]",'',item)
  item = re.sub(" ",'',item)
  return item
def acurracy_category(result, dataset, path):
  categories = np.unique(dataset["cat"])
  print(categories)
  result["cat"] = dataset["cat"]
  accuracy = {
      "cat":[],
      "sample" :[],
      "acc_1":[],
      "acc_3":[],
      "acc_5":[],
      "acc_10":[],
  }
  for cat in categories:
    acc_1 = 0
    acc_3 = 0
    acc_5 = 0
    acc_10 = 0
    df = result[result["cat"]==cat]
    # print(len(df))
    for index, row in tqdm(df.iterrows()):
      res = row["predicted_answer"]
      res = re.sub("\'",'',res)
      res = re.sub("\[",'',res)
      res = re.sub("\]",'',res)
      res = re.sub(" ",'',res)
      res = res.split(",")
      # res = [pre_proc(item) for item in res]
      ground_truth_response = row["ground_truth"]
      if res[0] == ground_truth_response:
        acc_1 += 1
        acc_3 += 1
        acc_5 += 1
        acc_10 += 1
      elif ground_truth_response in res[0:3]:
        acc_3 += 1
        acc_5 += 1
        acc_10 += 1
      elif ground_truth_response in res[0:5]:
        acc_5 += 1
        acc_10 += 1
      elif ground_truth_response in res:
        acc_10 += 1
    total = len(df)
    accuracy["cat"].append(cat)
    try:
      accuracy["sample"].append(df.iloc[0]['test'])
    except:
      accuracy["sample"].append(df.iloc[0]['test1'])
    accuracy["acc_1"].append(acc_1/total)
    accuracy["acc_3"].append(acc_3/total)
    accuracy["acc_5"].append(acc_5/total)
    accuracy["acc_10"].append(acc_10/total)
    print("\n category is : ", cat) 
    try:
      print("Sample test : {}".format(df.iloc[0]['test']))
    except:
      print("Sample test1 : {}".format(df.iloc[0]['test1']))
      print("Sample test2 : {}".format(df.iloc[0]['test2']))
    print("acc_1: ", acc_1/total) 
    print("acc_3: ", acc_3/total)
    print("acc_5: ", acc_5/total)
    print("acc_10: ", acc_10/total)
    print("===================================")
  accuracy = pd.DataFrame(accuracy)
  save_csv(ROOT+path, accuracy)
  print("file saved to {}".format(ROOT+path))

In [30]:
def semantic_acc(result, dataset,semantic): 
  categories = np.unique(result["cat"]) 
  length = 0 
  acc_1 = 0 
  acc_3 = 0 
  acc_5 = 0 
  acc_10 = 0 
  for sem in semantic: 
    size = len(dataset[dataset["cat"]==sem]) 
    # print(sem) 
    # print(size) 
    res = result[result["cat"]==sem] 
    # print(res.iloc[0]["acc_1"]) 
    length += size 
    acc_1 += res.iloc[0]["acc_1"] * size 
    acc_3 += res.iloc[0]["acc_3"] * size 
    acc_5 += res.iloc[0]["acc_5"] * size 
    acc_10 += res.iloc[0]["acc_10"] * size 
  acc_1 = acc_1/length 
  acc_3 = acc_3/length 
  acc_5 = acc_5/length 
  acc_10 = acc_10/length 
  print("semantic results") 
  print("length: ", length)  
  print("acc_1: ", acc_1)  
  print("acc_3: ", acc_3) 
  print("acc_5: ", acc_5) 
  print("acc_10: ", acc_10) 
  print("===================================") 
 
  length = 0 
  acc_1 = 0 
  acc_3 = 0 
  acc_5 = 0 
  acc_10 = 0 
  for ca in categories: 
    if ca not in semantic: 
      size = len(dataset[dataset["cat"]==ca]) 
      # print(ca) 
      # print(size) 
      length += size 
      res = result[result["cat"]==ca] 
      # print(res.iloc[0]["acc_1"]) 
      acc_1 += res.iloc[0]["acc_1"] * size 
      acc_3 += res.iloc[0]["acc_3"] * size 
      acc_5 += res.iloc[0]["acc_5"] * size 
    acc_10 += res.iloc[0]["acc_10"] * size 
  acc_1 = acc_1/length 
  acc_3 = acc_3/length 
  acc_5 = acc_5/length 
  acc_10 = acc_10/length 
  print("syntactic results") 
  print("length: ", length)  
  print("acc_1: ", acc_1)  
  print("acc_3: ", acc_3) 
  print("acc_5: ", acc_5) 
  print("acc_10: ", acc_10) 
  print("===================================")

# Data

analogy test 1

In [6]:
# df = read_csv('/content/drive/MyDrive/Colab Notebooks/nlp /analogy/analogy.csv')
df = read_csv('/content/drive/MyDrive/Analogy_NLP_Files/analogy.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  


The size of the analogy test set is : 19716
                 cat  first second    third   fourth
0  semantic-capitals  ایران  تهران      چین      پکن
1  semantic-capitals  ایران  تهران      هند  دهلی‌نو
2  semantic-capitals  ایران  تهران     ژاپن    توکیو
3  semantic-capitals  ایران  تهران  فیلیپین    مانیل
4  semantic-capitals  ایران  تهران    روسیه    موسکو


analogy farsi

In [29]:
# df_farsi = read_csv('/content/drive/MyDrive/Colab Notebooks/nlp /analogy/analogy_farsi.csv')
df_farsi = read_csv('/content/drive/MyDrive/Analogy_NLP_Files/analogy_farsi.csv')

The size of the analogy test set is : 30072
    cat first second     third   fourth
0  city   کرج  البرز     شیراز     فارس
1  city   کرج  البرز     اهواز  خوزستان
2  city   کرج  البرز       رشت    گیلان
3  city   کرج  البرز  بندرعباس  هرمزگان
4  city   کرج  البرز      اراک    مرکزی


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  


analogy slang

In [23]:
# df_slang= read_csv('/content/drive/MyDrive/Colab Notebooks/nlp /analogy/analogy_slang.csv')
df_slang= read_csv('/content/drive/MyDrive/Analogy_NLP_Files/analogy_slang.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  


The size of the analogy test set is : 2332
              cat  first second    third    fourth
0  semantic-slang  هیشکی  هیچکس      باس      باید
1  semantic-slang  هیشکی  هیچکس      باس    بایستی
2  semantic-slang  هیشکی  هیچکس      مگه       مگر
3  semantic-slang  هیشکی  هیچکس  رفتارای  رفتارهای
4  semantic-slang  هیشکی  هیچکس    کردین     کردید


bokai analogy

In [24]:
# df_bokai = txt_to_csv('/content/drive/MyDrive/Colab Notebooks/nlp /analogy/FA-GOOGLE-TABBED.txt',
                      # '/content/drive/MyDrive/Colab Notebooks/nlp /analogy/bokai.csv')

# v1
df_bokai = txt_to_csv('/content/drive/MyDrive/Analogy_NLP_Files/FA-GOOGLE-TABBED-normal.txt',
                       '/content/drive/MyDrive/Analogy_NLP_Files/bokai.csv')

# v2
# url = '/content/drive/MyDrive/Analogy_NLP_Files/FA-GOOGLE-TABBED.txt'
# url_2 = '/content/drive/MyDrive/Analogy_NLP_Files/FA-GOOGLE-TABBED.csv'
# df_bokai_2 = text_to_csv_2(url, url_2)
# df_bokai_2.head()


31347
                 cat first second  third  fourth
0  Adjective-Adverb:  سریع  سریعا  مخصوص  مخصوصا
1  Adjective-Adverb:  سریع  سریعا   کامل   کاملا
2  Adjective-Adverb:  سریع  سریعا  معمول  معمولا
3  Adjective-Adverb:  سریع  سریعا   اخیر   اخیرا
4  Adjective-Adverb:  سریع  سریعا  ناچار  ناچارا


sat analogy

In [25]:
# df_sat, df_sat_test = process_sat_analogy('/content/drive/MyDrive/Colab Notebooks/nlp /analogy/SAT_ANALOGY.csv',
                                          # '/content/drive/MyDrive/Colab Notebooks/nlp /analogy/SAT_ANALOGY_Process.csv')

df_sat, df_sat_test = process_sat_analogy('/content/drive/MyDrive/Analogy_NLP_Files/SAT_ANALOGY.csv',
                                           '/content/drive/MyDrive/Analogy_NLP_Files/SAT_ANALOGY_Process.csv')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.




The size of the analogy test set is : 390
   ID                                       Desc  ... Unnamed: 4 Unnamed: 5
0   1  Class Inclusion  Taxonomic  emotion:rage   ...        NaN        NaN
1   1  Class Inclusion  Taxonomic  emotion:rage   ...        NaN        NaN
2   1  Class Inclusion  Taxonomic  emotion:rage   ...        NaN        NaN
3   1  Class Inclusion  Taxonomic  emotion:rage   ...        NaN        NaN
4   1  Class Inclusion  Taxonomic  emotion:rage   ...        NaN        NaN

[5 rows x 6 columns]
Number of unique description in sat test : 67


In [26]:
df_sat_test.head()

,cat,first,second,third,fourth
0,Attribute Agent Attribute:State (adjective:no...,فقیر,بیپولی,ملتهب,دعوا
1,Attribute Agent Attribute:State (adjective:no...,فقیر,بیپولی,مضطرب,بیقرار
2,Attribute Agent Attribute:State (adjective:no...,فقیر,بیپولی,منفک,تنها
3,Attribute Agent Attribute:State (adjective:no...,ملتهب,دعوا,مضطرب,بیقرار
4,Attribute Agent Attribute:State (adjective:no...,ملتهب,دعوا,منفک,تنها


# BERT Model

In [ ]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
import logging
logging.basicConfig(level=logging.INFO)# OPTIONAL

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

!nvidia-smi

Sun Feb 20 13:41:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
MODEL_NAME= 'HooshvareLab/bert-fa-base-uncased'

OUTPUT_PATH = '/content/drive/MyDrive/Colab Notebooks/nlp/digikala/analogy_model.pt'
# OUTPUT_PATH = '/content/drive/MyDrive/Analogy_NLP_Files/analogy_model.pt'

In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForMaskedLM.from_pretrained(MODEL_NAME)
model = model.to(device)
model.eval()

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=T

In [ ]:
# def predict_masked_sent(text, top_k=5):
#     # Tokenize input
#     text = "[CLS] %s [SEP]"%text
#     tokenized_text = tokenizer.tokenize(text)
#     masked_index = tokenized_text.index("[MASK]")
#     indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
#     tokens_tensor = torch.tensor([indexed_tokens])
#     tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

#     # Predict all tokens
#     with torch.no_grad():
#         outputs = model(tokens_tensor)
#         predictions = outputs[0]

#     probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
#     top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

#     for i, pred_idx in enumerate(top_k_indices):
#         predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
#         token_weight = top_k_weights[i]
#         print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))


In [ ]:
def predict_masked_sent_2(text, top_k=5):
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    results = []
    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        results.append(predicted_token)

    return results

In [ ]:
def predict_analogy(dataset,dataset_response):
  acc_1 = 0
  acc_3 = 0
  acc_5 = 0
  acc_10 = 0
  total_retrieved_result = []
  print(len(dataset))
  for i in tqdm(range(len(dataset))):
    result = predict_masked_sent_2(dataset[i], top_k=10)
    total_retrieved_result.append(result)
    if result[0]== dataset_response[i]:
      acc_1 += 1
      acc_3 += 1
      acc_5 += 1
      acc_10 += 1
    elif dataset_response[i] in result[0:3]:
      acc_3 += 1
      acc_5 += 1
      acc_10 += 1
    elif dataset_response[i] in result[0:5]:
      acc_5 += 1
      acc_10 += 1
    elif dataset_response[i] in result:
      acc_10 += 1
  total = len(dataset)
  print("\nacc_1: ", acc_1/total) 
  print("acc_3: ", acc_3/total)
  print("acc_5: ", acc_5/total)
  print("acc_10: ", acc_10/total)
  print("===================================")
  return total_retrieved_result

In [ ]:
def generate_pattern(dataset, pattern):
  pattern = re.sub("_","", pattern)
  pattern = pattern.split(" ")
  pattern = [p for p in pattern if p != ""]
  print("Pattern is : {}".format(pattern))
  # for p in pattern :
  #   print(p)
  dataset_1 = []
  dataset_1_responnse = []
  for index, row in dataset.iterrows():
    if "پایتخت" in pattern  :
      text = row["second"] +" " + pattern[0] +" "+ row["first"] + " " + pattern[1]+ ' ' + pattern[2]+ ' ' + pattern[3] +" " + pattern[4] + " " + row["third"] + " " + pattern[5]
    elif "مرکز" in pattern :
       text = row["first"] +" " + pattern[0] +" " +pattern[1] +" "+ row["second"] + " " + pattern[2]+ ' ' + pattern[3]+ ' ' +row["third"]+" "+ pattern[4] +" " + pattern[5] + " " + pattern[6] + " " + pattern[7]
    elif "همانطور" in pattern :
      text = pattern[0] +" " +pattern[1] +" " + row["second"] +" " + pattern[2]+" " +  row["first"] + " " + pattern[3]+ ' ' + pattern[4]+ ' ' + pattern[5] +" "  + row["third"] + " " + pattern[6]
    elif "عامیانه" in pattern :
      text = row["first"] +" " + pattern[0] +" " +pattern[1] +" "+ row["second"] + " " + pattern[2]+ ' ' + pattern[3]+ ' ' + row["third"] +" "+ pattern[4] +" "+ pattern[5] + " " + pattern[6] + " " + pattern[7]
    else :
      text = row["first"] +" " + pattern[0] +" "+ row["second"] + " " + pattern[1]+ ' ' + row["third"] + " "+ pattern[2] +" " + pattern[3] + " " + pattern[4]
    dataset_1.append(text)
    dataset_1_responnse.append(row["fourth"])
  print("Sample test : {} ".format(dataset_1[0]))
  print("Sample test answer is : {} ".format(dataset_1_responnse[0]))
  return dataset_1, dataset_1_responnse

## New Method

In [ ]:
def new_prediction_method(dataset): 
  acc_1 = 0 
  acc_3 = 0 
  acc_5 = 0 
  acc_10 = 0 
  total_retrieved_result = [] 
  total_relation_test1 = [] # top 10 relation ro negah midare 
  total_relation = [] # avalin relation ro negah midare 
  total_created_tests1 = [] 
  total_created_tests = [] 
  total_ground_truth_responses = [] 
  print(len(dataset)) 
  for index, row in tqdm(dataset.iterrows()): 
      test_1 = generate_first_pattern(row["first"], row["second"]) # تهران ماسک ایران است 
      # print(test_1) 
      relation = predict_masked_sent_2(test_1, top_k=10)  # predict mask 
      total_relation_test1.append(relation) 
      total_relation.append(relation[0]) 
      test_2 = generate_second_pattern(row["third"], relation[0]) # پایتخت چین ماسک است 
      total_created_tests1.append(test_1) 
      total_created_tests.append(test_2) 
      total_ground_truth_responses.append(row["fourth"]) 
      # print(test_2) 
      result = predict_masked_sent_2(test_2, top_k=10) 
      total_retrieved_result.append(result) 
      ground_truth_response = row["fourth"] 
      if result[0] == ground_truth_response: 
        acc_1 += 1 
        acc_3 += 1 
        acc_5 += 1 
        acc_10 += 1 
      elif ground_truth_response in result[0:3]: 
        acc_3 += 1 
        acc_5 += 1 
        acc_10 += 1 
      elif ground_truth_response in result[0:5]: 
        acc_5 += 1 
        acc_10 += 1 
      elif ground_truth_response in result: 
        acc_10 += 1 
  total = len(dataset) 
  print("\nacc_1: ", acc_1/total)  
  print("acc_3: ", acc_3/total) 
  print("acc_5: ", acc_5/total) 
  print("acc_10: ", acc_10/total) 
  print("===================================") 
  return total_retrieved_result, total_created_tests1, total_created_tests, total_ground_truth_responses, total_relation_test1, total_relation

def generate_first_pattern(word1, word2):
  return word2 +" " + '[MASK]' + ' '+ word1 +" " + "است"

def generate_second_pattern(word, relation):
  return relation + " " + word + " " +"[MASK]" +" " + "است"

# Predict

In [ ]:
def visualize(result, dataset, dataset_response):
  start = np.random.randint(low = 0,high = len(dataset)-10)
  print(start)
  for res, data, response in tqdm(zip(result[start:start + 5],dataset[start:start + 5],dataset_response[start:start + 5])):
    print("The test is : {}".format(data))
    print("The answer is : {}".format(response))
    print("The predicted result is : \n{}".format(res))
    print("\n")

In [32]:
# ROOT = "/content/drive/MyDrive/Colab Notebooks/nlp /analogy/"
ROOT = "/content/drive/MyDrive/Analogy_NLP_Files/"

## Save results

In [ ]:
def save_result(result, dataset, dataset_response, path):
  result_df = {"test":[],
          "predicted_answer":[],
          "ground_truth":[]}
  for res, data, response in tqdm(zip(result, dataset, dataset_response)):
    result_df["test"].append(data)
    result_df["ground_truth"].append(response)
    result_df["predicted_answer"].append(res)
  result_df = pd.DataFrame(result_df)
  save_csv(ROOT+path, result_df)
  print("file saved to {}".format(ROOT+path))

# save result of the new method
def save_result_new(result, tests1,tests2,response,total_relation, relation, path): 
  result_df = {"test2":[], 
               "test1":[], 
               "total_relation":[], 
               "relation":[], 
          "predicted_answer":[], 
          "ground_truth":[]} 
  for res, t1,t2, truth, total_rel, rel in tqdm(zip(result, tests1,tests2,response,total_relation, relation)): 
    result_df["test1"].append(t1) 
    result_df["test2"].append(t2) 
    result_df["total_relation"].append(total_rel) 
    result_df["relation"].append(rel) 
    result_df["ground_truth"].append(truth) 
    result_df["predicted_answer"].append(res) 
  result_df = pd.DataFrame(result_df) 
  save_csv(ROOT+path, result_df) 
  print("file saved to {}".format(ROOT+path))

In [ ]:
def relation_by_cat(result, dataset, path): 
  categories = np.unique(dataset["cat"]) 
  print(categories) 
  result["cat"] = dataset["cat"] 
  summary = { 
      "cat":[], 
      "rel" :[], 
      "top10":[], 
  } 
  for cat in categories: 
    df = result[result["cat"]==cat] 
    rel = [] 
    top10 = [] 
    for index, row in tqdm(df.iterrows()):  
      rel.append(row["relation"]) 
      res = row["total_relation"] 
      res = re.sub("\'",'',res) 
      res = re.sub("\[",'',res) 
      res = re.sub("\]",'',res) 
      res = re.sub(" ",'',res) 
      res = res.split(",") 
      for r in res: 
        top10.append(r) 
    rel =list(set(rel)) 
    top10 = list(set(top10)) 
    summary["rel"].append(rel) 
    summary["top10"].append(top10) 
    summary["cat"].append(cat) 
  sum = pd.DataFrame(summary) 
  save_csv(ROOT+path, sum)

In [ ]:
pattern = ["_ به _ مانند _ به [MASK] است",
           "_ پایتخت _ است و [MASK] پایتخت _ است",
           "_ مرکز استان _ است و _ مرکز استان [MASK] است",
           "همانطور که _ برای _ است [MASK] برای _ است",
           "_ معادل عامیانه _ است و _ معادل عامیانه [MASK] است",
           ]


## analogy 1

In [ ]:
df_dataset, df_response = generate_pattern(df, pattern[0])

Pattern is : ['به', 'مانند', 'به', '[MASK]', 'است']
Sample test : ایران به تهران مانند چین به [MASK] است 
Sample test answer is : پکن 


In [ ]:
result_analogy = predict_analogy(df_dataset, df_response)

19716


100%|██████████| 19716/19716 [04:21<00:00, 75.41it/s]


acc_1:  0.016889835666463786
acc_3:  0.057567457902211405
acc_5:  0.08632582673970379
acc_10:  0.13390139987827146


In [ ]:
visualize(result_analogy, df_dataset, df_response)

1763


5it [00:00, 10048.64it/s]

The test is : کره‌شمالی به پیونگ‌یانگ مانند ترکمنستان به [MASK] است
The answer is : عشق‌آباد
The predicted result is : 
['کره', 'ایران', 'کرهشمالی', 'پیونگیانگ', 'چین', 'روسیه', 'معنی', 'اشتباه', 'نزدیک', 'بخارا']


The test is : کره‌شمالی به پیونگ‌یانگ مانند آمریکا به [MASK] است
The answer is : واشینگتن
The predicted result is : 
['عقب', 'فکر', 'اشتباه', 'جنگ', 'کره', 'جلو', 'دروغ', 'راه', 'گذشته', 'روز']


The test is : کره‌شمالی به پیونگ‌یانگ مانند امارات به [MASK] است
The answer is : ابوظبی
The predicted result is : 
['کره', 'ایران', 'چین', 'عربستان', 'روز', 'عقب', 'خاورمیانه', 'فارس', 'ژاپن', 'قطر']


The test is : کره‌شمالی به پیونگ‌یانگ مانند تاجیکستان به [MASK] است
The answer is : دوشنبه
The predicted result is : 
['اشتباه', 'کره', 'نزدیک', 'عقب', 'نزدیکتر', 'ایران', 'کرهشمالی', 'معنی', 'چین', 'معروف']


The test is : کره‌شمالی به پیونگ‌یانگ مانند بحرین به [MASK] است
The answer is : منامه
The predicted result is : 
['کره', 'ایران', 'اشتباه', 'کرهشمالی', 'نزدیک', 'عقب', 'پیونگیا

In [ ]:
save_result(result_analogy, df_dataset, df_response, "analogy_test1_1.csv")

19716it [00:00, 1360852.07it/s]

file saved to /content/drive/MyDrive/Analogy_NLP_Files/analogy_test1_1.csv


In [ ]:
df_result = pd.read_csv(ROOT+"analogy_test1_1.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df,"cat_analogy_test1_1.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['semantic-Whole to part' 'semantic-capitals' 'semantic-currency'
 'semantic-family' 'syntactic-Comparative Adjectives'
 'syntactic-Superlative Adjectives' 'syntactic-antonym' 'syntactic-verb']


420it [00:00, 10353.75it/s]



 category is :  semantic-Whole to part
Sample test : آب به قطره مانند بدن به [MASK] است
acc_1:  0.03571428571428571
acc_3:  0.2261904761904762
acc_5:  0.3238095238095238
acc_10:  0.3952380952380952


4692it [00:00, 10243.12it/s]


...


 category is :  syntactic-Superlative Adjectives
Sample test : خوب به خوب‌ترین مانند بد به [MASK] است
acc_1:  0.0
acc_3:  0.0025399129172714078
acc_5:  0.0047169811320754715
acc_10:  0.014876632801161103


506it [00:00, 8625.77it/s]



 category is :  syntactic-antonym
Sample test : نامشخص به مشخص مانند نامرتب به [MASK] است
acc_1:  0.0
acc_3:  0.041501976284584984
acc_5:  0.07509881422924901
acc_10:  0.1482213438735178


1980it [00:00, 9111.40it/s]


 category is :  syntactic-verb
Sample test : می‌رود به رفت مانند می‌خورد به [MASK] است
acc_1:  0.000505050505050505
acc_3:  0.0065656565656565654
acc_5:  0.011616161616161616
acc_10:  0.025252525252525252
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_analogy_test1_1.csv


In [ ]:
# np.unique(df["cat"])

In [ ]:
# df_dataset2, df_response2 = generate_pattern(df[df["cat"]=='semantic-capitals'], pattern[1])

In [ ]:
# result_analogy2 = predict_analogy(df_dataset2, df_response2)

In [ ]:
# visualize(result_analogy2,df_dataset2, df_response2)

In [ ]:
df_dataset3, df_response3 = generate_pattern(df, pattern[3])

Pattern is : ['همانطور', 'که', 'برای', 'است', '[MASK]', 'برای', 'است']
Sample test : همانطور که تهران برای ایران است [MASK] برای چین است 
Sample test answer is : پکن 


In [ ]:
result_analogy3 = predict_analogy(df_dataset3, df_response3)

19716


100%|██████████| 19716/19716 [04:56<00:00, 66.57it/s]


acc_1:  0.01577399066747819
acc_3:  0.051075268817204304
acc_5:  0.10204909717995536
acc_10:  0.16357273280584297


In [ ]:
visualize(result_analogy3,df_dataset3, df_response3 )

12802


5it [00:00, 9023.89it/s]

The test is : همانطور که نازکتر برای نازک است [MASK] برای کثیف است
The answer is : کثیف‌تر
The predicted result is : 
['نازکتر', 'و', '،', 'باریکتر', 'سختتر', 'کوچکتر', 'بزرگتر', 'بیشتر', 'بلندتر', 'ضخیمتر']


The test is : همانطور که نازکتر برای نازک است [MASK] برای بلند است
The answer is : بلندتر
The predicted result is : 
['نازکتر', 'بلندتر', 'باریکتر', 'و', 'کوتاهتر', '،', 'کوچکتر', 'طولانیتر', 'بیشتر', 'بزرگتر']


The test is : همانطور که نازکتر برای نازک است [MASK] برای کوتاه است
The answer is : کوتاه‌تر
The predicted result is : 
['نازکتر', 'و', 'بلندتر', 'باریکتر', '،', 'کوتاهتر', 'نازک', 'طولانیتر', 'کوچکتر', 'بیشتر']


The test is : همانطور که نازکتر برای نازک است [MASK] برای چاق است
The answer is : چاق‌تر
The predicted result is : 
['نازکتر', 'باریکتر', '،', 'بلندتر', 'و', 'نازک', 'کوچکتر', 'بیشتر', 'ضخیمتر', 'بزرگتر']


The test is : همانطور که نازکتر برای نازک است [MASK] برای لاغر است
The answer is : لاغرتر
The predicted result is : 
['نازکتر', 'باریکتر', '،', 'و', 'بلندتر

In [ ]:
save_result(result_analogy3,df_dataset3, df_response3 ,"analogy_test1_2.csv")

19716it [00:00, 825496.10it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/analogy_test1_2.csv


In [ ]:
df_result = pd.read_csv(ROOT+"analogy_test1_2.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df,"cat_analogy_test1_2.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['semantic-Whole to part' 'semantic-capitals' 'semantic-currency'
 'semantic-family' 'syntactic-Comparative Adjectives'
 'syntactic-Superlative Adjectives' 'syntactic-antonym' 'syntactic-verb']


420it [00:00, 4309.41it/s]



 category is :  semantic-Whole to part
Sample test : همانطور که قطره برای آب است [MASK] برای بدن است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.016666666666666666
acc_10:  0.04523809523809524


4692it [00:00, 4981.76it/s]


...


 category is :  syntactic-Superlative Adjectives
Sample test : همانطور که خوب‌ترین برای خوب است [MASK] برای بد است
acc_1:  0.0039912917271407835
acc_3:  0.007619738751814224
acc_5:  0.018867924528301886
acc_10:  0.040638606676342524


506it [00:00, 2285.36it/s]



 category is :  syntactic-antonym
Sample test : همانطور که مشخص برای نامشخص است [MASK] برای نامرتب است
acc_1:  0.0
acc_3:  0.003952569169960474
acc_5:  0.01383399209486166
acc_10:  0.04743083003952569


1980it [00:00, 3613.59it/s]



 category is :  syntactic-verb
Sample test : همانطور که رفت برای می‌رود است [MASK] برای می‌خورد است
acc_1:  0.0
acc_3:  0.000505050505050505
acc_5:  0.0015151515151515152
acc_10:  0.0035353535353535356
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_analogy_test1_2.csv


## with new method

In [ ]:
analogy_result4,analogy_tests1, analogy_tests2, analogy_responses4, total_relation, relation = new_prediction_method(df)

19716


19716it [16:20, 20.10it/s]


acc_1:  0.16235544735240415
acc_3:  0.18548387096774194
acc_5:  0.19263542300669506
acc_10:  0.2141407993507811


In [ ]:
save_result_new(analogy_result4, analogy_tests1,analogy_tests2,analogy_responses4,total_relation,relation, "new_method_analogy.csv")

19716it [00:00, 689980.87it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/new_method_analogy.csv


In [ ]:
df_result4 = pd.read_csv(ROOT+"new_method_analogy.csv", engine='python',encoding='utf-8', error_bad_lines=False)
df_result4.head()
acurracy_category(df_result4, df,"cat_new_method_analogy.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['semantic-Whole to part' 'semantic-capitals' 'semantic-currency'
 'semantic-family' 'syntactic-Comparative Adjectives'
 'syntactic-Superlative Adjectives' 'syntactic-antonym' 'syntactic-verb']


420it [00:00, 10137.70it/s]



 category is :  semantic-Whole to part
Sample test1 : قطره [MASK] آب است
Sample test2 : دیگر بدن [MASK] است
acc_1:  0.011904761904761904
acc_3:  0.023809523809523808
acc_5:  0.023809523809523808
acc_10:  0.030952380952380953


4692it [00:00, 10560.48it/s]


...


 category is :  syntactic-Superlative Adjectives
Sample test1 : خوب‌ترین [MASK] خوب است
Sample test2 : کار بد [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


506it [00:00, 9741.30it/s]



 category is :  syntactic-antonym
Sample test1 : مشخص [MASK] نامشخص است
Sample test2 : نیست نامرتب [MASK] است
acc_1:  0.0
acc_3:  0.037549407114624504
acc_5:  0.043478260869565216
acc_10:  0.09683794466403162


1980it [00:00, 9773.29it/s]



 category is :  syntactic-verb
Sample test1 : رفت [MASK] می‌رود است
Sample test2 : و می‌خورد [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_new_method_analogy.csv


In [ ]:
df_analogy_result4 = pd.read_csv(ROOT+"new_method_analogy.csv", engine='python',encoding='utf-8', error_bad_lines=False) 
df_analogy_result4.head() 
relation_by_cat(df_analogy_result4, df, "analogy_cat_rel.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['semantic-Whole to part' 'semantic-capitals' 'semantic-currency'
 'semantic-family' 'syntactic-Comparative Adjectives'
 'syntactic-Superlative Adjectives' 'syntactic-antonym' 'syntactic-verb']


420it [00:00, 4879.53it/s]
4692it [00:01, 4209.25it/s]
6006it [00:01, 4443.04it/s]
600it [00:00, 3535.71it/s]
2756it [00:00, 4543.74it/s]
2756it [00:00, 4552.84it/s]
506it [00:00, 4629.46it/s]
1980it [00:00, 4707.22it/s]


In [35]:
sem_analogy = ['semantic-Whole to part' ,'semantic-capitals' ,'semantic-currency','semantic-family']
df_analogy_result = pd.read_csv(ROOT+"cat_new_method_analogy.csv", engine='python',encoding='utf-8', error_bad_lines=False) 
semantic_acc(df_analogy_result, df, sem_analogy)

semantic results
length:  11718
acc_1:  0.2715480457415941
acc_3:  0.3072196620583717
acc_5:  0.3143027820447175
acc_10:  0.3466461853558628
syntactic results
length:  7998
acc_1:  0.002375593898474607
acc_3:  0.007126781695423831
acc_5:  0.014378594648662134
acc_10:  0.03700925231307822


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


## analogy farsi

In [ ]:
dataset_farsi, dataset_farsi_response = generate_pattern(df_farsi, pattern[0])

Pattern is : ['به', 'مانند', 'به', '[MASK]', 'است']
Sample test : کرج به البرز مانند شیراز به [MASK] است 
Sample test answer is : فارس 


In [ ]:
result_farsi = predict_analogy(dataset_farsi, dataset_farsi_response)

30072


100%|██████████| 30072/30072 [06:19<00:00, 79.15it/s]


acc_1:  0.05510109071561586
acc_3:  0.1278265496142591
acc_5:  0.17218675179569035
acc_10:  0.23540170258047352


In [ ]:
visualize(result_farsi,dataset_farsi, dataset_farsi_response)

22493


5it [00:00, 9210.15it/s]

The test is : مناسب به نامناسب مانند منظم به [MASK] است
The answer is : نامنظم
The predicted result is : 
['پایین', 'بالا', 'صورت', 'مناسب', 'اطراف', 'دور', 'ارامی', 'خوب', 'اهستگی', 'زیر']


The test is : مناسب به نامناسب مانند موزون به [MASK] است
The answer is : ناموزون
The predicted result is : 
['پایین', 'موزون', 'کف', 'بالا', 'نفس', 'صورت', 'دقیقه', 'صفر', 'زمان', 'چپ']


The test is : مناسب به نامناسب مانند منسجم به [MASK] است
The answer is : نامنسجم
The predicted result is : 
['پایین', 'مناسب', 'خوبی', 'خوب', 'بالا', 'صورت', 'ان', 'غلط', 'هم', 'اطراف']


The test is : مناسب به نامناسب مانند سازگار به [MASK] است
The answer is : ناسازگار
The predicted result is : 
['پایین', 'بد', 'خوب', 'نامناسب', 'مناسب', 'هوا', 'بالا', 'خوبی', 'سبک', 'پايین']


The test is : مناسب به نامناسب مانند مشروع به [MASK] است
The answer is : نامشروع
The predicted result is : 
['نامناسب', 'نادرست', 'بد', 'مناسب', 'غلط', 'مطلوب', 'پایین', 'خوب', 'اشتباه', 'صحیح']




In [ ]:
save_result(result_farsi,dataset_farsi, dataset_farsi_response, "farsi_1.csv")

30072it [00:00, 1378346.50it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/farsi_1.csv


In [ ]:
df_result = pd.read_csv(ROOT+"farsi_1.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df_farsi,"cat_farsi_1.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['capital' 'city' 'currency' 'family' 'gram_adj2adv' 'gram_antonym'
 'gram_comparative' 'gram_firstperson' 'gram_nationality' 'gram_noun2adv'
 'gram_past' 'gram_plural' 'gram_presentparticiple' 'gram_superlative'
 'gram_thirdperson']


5402it [00:01, 5391.75it/s]



 category is :  capital
Sample test : چین به پکن مانند ژاپن به [MASK] است
acc_1:  0.010181414291003332
acc_3:  0.06293965198074787
acc_5:  0.1036653091447612
acc_10:  0.17604590892262126


7832it [00:00, 10260.04it/s]


...


 category is :  gram_presentparticiple
Sample test : رقصیدن به می‌رقصد مانند بلعیدن به [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


1it [00:00, 1683.11it/s]



 category is :  gram_superlative
Sample test : سریع به سریعترین مانند بزرگ به [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


2591it [00:00, 8712.10it/s]



 category is :  gram_thirdperson
Sample test : رفت به رفتند مانند رقصید به [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.00038595137012736397
acc_10:  0.00038595137012736397
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_farsi_1.csv


In [ ]:
# np.unique(df_farsi["cat"])

In [ ]:
# dataset_farsi2, dataset_farsi_response2 = generate_pattern(df_farsi[df_farsi["cat"]=='city'], pattern[2])

In [ ]:
# result_farsi2 = predict_analogy(dataset_farsi2, dataset_farsi_response2)

In [ ]:
# visualize(result_farsi2, dataset_farsi2, dataset_farsi_response2)

In [ ]:
dataset_farsi3, dataset_farsi_response3 = generate_pattern(df_farsi, pattern[3])

Pattern is : ['همانطور', 'که', 'برای', 'است', '[MASK]', 'برای', 'است']
Sample test : همانطور که البرز برای کرج است [MASK] برای شیراز است 
Sample test answer is : فارس 


In [ ]:
result_farsi3 = predict_analogy(dataset_farsi3, dataset_farsi_response3)

30072


100%|██████████| 30072/30072 [06:14<00:00, 80.31it/s]


acc_1:  0.04891593508911945
acc_3:  0.1293562117584464
acc_5:  0.20374434690077148
acc_10:  0.28900638467677575


In [ ]:
visualize(result_farsi3, dataset_farsi3, dataset_farsi_response3)

3849


5it [00:00, 9850.41it/s]

The test is : همانطور که بوشهر برای برازجان است [MASK] برای خرمدره است
The answer is : زنجان
The predicted result is : 
['اصفهان', 'بوشهر', 'شیراز', 'اهواز', '،', 'کرمان', 'تهران', 'خوزستان', 'رشت', 'خرمشهر']


The test is : همانطور که بوشهر برای برازجان است [MASK] برای جوانرود است
The answer is : کرمانشاه
The predicted result is : 
['،', 'اصفهان', 'کرمانشاه', 'قدس', 'اهواز', 'بوشهر', 'جوانرود', 'و', 'برازجان', 'شیراز']


The test is : همانطور که بوشهر برای برازجان است [MASK] برای هشتپر است
The answer is : گیلان
The predicted result is : 
['اصفهان', 'رشت', 'بوشهر', 'اهواز', '،', 'شیراز', 'تهران', 'و', 'کرمان', 'انزلی']


The test is : همانطور که بوشهر برای برازجان است [MASK] برای حمیدیا است
The answer is : یزد
The predicted result is : 
['بوشهر', 'اصفهان', 'شیراز', 'برازجان', 'اهواز', '،', 'کرمان', 'و', 'فارس', 'تهران']


The test is : همانطور که بوشهر برای برازجان است [MASK] برای کنگاور است
The answer is : کرمانشاه
The predicted result is : 
['کرمانشاه', 'اصفهان', 'کنگاور', '،', 'شیرا

In [ ]:
save_result(result_farsi3, dataset_farsi3, dataset_farsi_response3, "farsi_2.csv")

30072it [00:00, 1432608.04it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/farsi_2.csv


In [ ]:
df_result = pd.read_csv(ROOT+"farsi_2.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df_farsi,"cat_farsi_2.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['capital' 'city' 'currency' 'family' 'gram_adj2adv' 'gram_antonym'
 'gram_comparative' 'gram_firstperson' 'gram_nationality' 'gram_noun2adv'
 'gram_past' 'gram_plural' 'gram_presentparticiple' 'gram_superlative'
 'gram_thirdperson']


5402it [00:01, 4921.22it/s]



 category is :  capital
Sample test : همانطور که پکن برای چین است [MASK] برای ژاپن است
acc_1:  0.057201036653091446
acc_3:  0.1691965938541281
acc_5:  0.3122917437985931
acc_10:  0.4378008145131433


7832it [00:00, 10548.55it/s]


...


 category is :  gram_presentparticiple
Sample test : همانطور که می‌رقصد برای رقصیدن است [MASK] برای بلعیدن است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


1it [00:00, 2047.00it/s]



 category is :  gram_superlative
Sample test : همانطور که سریعترین برای سریع است [MASK] برای بزرگ است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


2591it [00:00, 8860.21it/s]



 category is :  gram_thirdperson
Sample test : همانطور که رفتند برای رفت است [MASK] برای رقصید است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_farsi_2.csv


## with new method

In [ ]:
fasri_result4,fasri_tests1, fasri_tests2, fasri_responses4, total_relation_2, relation_2  = new_prediction_method(df_farsi)

30072


30072it [23:45, 21.10it/s]


acc_1:  0.12922319765895185
acc_3:  0.14847698856078745
acc_5:  0.15765496142591115
acc_10:  0.18452380952380953


In [ ]:
save_result_new(fasri_result4,fasri_tests1, fasri_tests2, fasri_responses4,total_relation_2, relation_2, "new_method_farsi.csv")

30072it [00:00, 773900.70it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/new_method_farsi.csv


In [ ]:
df_farsi_result4 = pd.read_csv(ROOT+"new_method_farsi.csv", engine='python',encoding='utf-8', error_bad_lines=False)
df_farsi_result4.head()
acurracy_category(df_farsi_result4, df_farsi,"cat_new_method_farsi.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['capital' 'city' 'currency' 'family' 'gram_adj2adv' 'gram_antonym'
 'gram_comparative' 'gram_firstperson' 'gram_nationality' 'gram_noun2adv'
 'gram_past' 'gram_plural' 'gram_presentparticiple' 'gram_superlative'
 'gram_thirdperson']


5402it [00:00, 9943.13it/s] 



 category is :  capital
Sample test1 : پکن [MASK] چین است
Sample test2 : پایتخت ژاپن [MASK] است
acc_1:  0.6601258793039615
acc_3:  0.7217697149203999
acc_5:  0.734727878563495
acc_10:  0.7719363198815253


7832it [00:00, 10142.62it/s]


...


 category is :  gram_presentparticiple
Sample test1 : می‌رقصد [MASK] رقصیدن است
Sample test2 : ، بلعیدن [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


1it [00:00, 1822.03it/s]



 category is :  gram_superlative
Sample test1 : سریعترین [MASK] سریع است
Sample test2 : قطار بزرگ [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


2591it [00:00, 9315.15it/s] 



 category is :  gram_thirdperson
Sample test1 : رفتند [MASK] رفت است
Sample test2 : و رقصید [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_new_method_farsi.csv


In [ ]:
df_farsi_result4 = pd.read_csv(ROOT+"new_method_farsi.csv", engine='python',encoding='utf-8', error_bad_lines=False) 
df_farsi_result4.head() 
relation_by_cat(df_farsi_result4, df_farsi, "farsi_cat_rel.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['capital' 'city' 'currency' 'family' 'gram_adj2adv' 'gram_antonym'
 'gram_comparative' 'gram_firstperson' 'gram_nationality' 'gram_noun2adv'
 'gram_past' 'gram_plural' 'gram_presentparticiple' 'gram_superlative'
 'gram_thirdperson']


5402it [00:00, 10387.68it/s]
7832it [00:00, 10267.24it/s]
1260it [00:00, 10376.46it/s]
342it [00:00, 8064.79it/s]
1332it [00:00, 10960.77it/s]
1260it [00:00, 10680.62it/s]
2519it [00:00, 10650.83it/s]
1260it [00:00, 9990.31it/s] 
1406it [00:00, 9599.17it/s] 
1056it [00:00, 9376.79it/s]
1260it [00:00, 10691.21it/s]
2550it [00:00, 9956.25it/s] 
1it [00:00, 1108.72it/s]
1it [00:00, 1833.17it/s]
2591it [00:00, 9884.98it/s]


In [33]:
sem_farsi = ['capital','city','currency','family']
df_farsi_result = pd.read_csv(ROOT+"cat_new_method_farsi.csv", engine='python',encoding='utf-8', error_bad_lines=False) 
semantic_acc(df_farsi_result, df_farsi, sem_farsi)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


semantic results
length:  14836
acc_1:  0.2549204637368563
acc_3:  0.29010514963602047
acc_5:  0.3021703963332434
acc_10:  0.34787004583445674
syntactic results
length:  15236
acc_1:  0.006825938566552897
acc_3:  0.0105670779732213
acc_5:  0.016933578367025456
acc_10:  0.029404043055920166


## analoggy slang

In [ ]:
dataset_slang, dataset_slang_response = generate_pattern(df_slang, pattern[0])

Pattern is : ['به', 'مانند', 'به', '[MASK]', 'است']
Sample test : هیشکی به هیچکس مانند باس به [MASK] است 
Sample test answer is : باید 


In [ ]:
result_slang = predict_analogy(dataset_slang, dataset_slang_response)

2332


100%|██████████| 2332/2332 [00:29<00:00, 80.37it/s]


acc_1:  0.001286449399656947
acc_3:  0.007718696397941681
acc_5:  0.012435677530017153
acc_10:  0.02358490566037736


In [ ]:
visualize(result_slang,dataset_slang, dataset_slang_response )

1030


5it [00:00, 10496.26it/s]

The test is : بازم به بازهم مانند خودمون به [MASK] است
The answer is : خودمان
The predicted result is : 
['عقب', 'جلو', 'نظر', 'گذشته', 'روز', 'تو', 'راه', 'رو', 'معنی', 'خودمون']


The test is : بازم به بازهم مانند خونمون به [MASK] است
The answer is : خانه‎مان
The predicted result is : 
['عقب', 'جلو', 'اینده', 'جهنم', 'تو', 'گذشته', 'نظر', 'پایان', 'بالا', 'بیرون']


The test is : بازم به بازهم مانند دخترا به [MASK] است
The answer is : دخترها
The predicted result is : 
['عقب', 'دنبالش', 'جلو', 'گذشته', 'تو', 'برگشته', 'نظر', 'پایان', 'من', 'اینده']


The test is : بازم به بازهم مانند پسرا به [MASK] است
The answer is : پسرها
The predicted result is : 
['عقب', 'جلو', 'دنبالش', 'برگشته', 'تو', 'باز', 'گذشته', 'پایان', 'بعد', 'من']


The test is : بازم به بازهم مانند زنا به [MASK] است
The answer is : زنها
The predicted result is : 
['پایان', 'عقب', 'کار', 'نفس', 'جلو', 'صورت', 'بار', 'اخر', 'سر', 'جهنم']




In [ ]:
save_result(result_slang, dataset_slang, dataset_slang_response, "slang_1.csv")

2332it [00:00, 941850.45it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/slang_1.csv


In [ ]:
df_result = pd.read_csv(ROOT+"slang_1.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df_slang,"cat_slang_1.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['semantic-slang']


2332it [00:00, 4444.07it/s]



 category is :  semantic-slang
Sample test : هیشکی به هیچکس مانند باس به [MASK] است
acc_1:  0.001286449399656947
acc_3:  0.007718696397941681
acc_5:  0.012435677530017153
acc_10:  0.02358490566037736
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_slang_1.csv


In [ ]:
dataset_slang2, dataset_slang_response2 = generate_pattern(df_slang, pattern[4])

Pattern is : ['معادل', 'عامیانه', 'است', 'و', 'معادل', 'عامیانه', '[MASK]', 'است']
Sample test : هیشکی معادل عامیانه هیچکس است و باس معادل عامیانه [MASK] است 
Sample test answer is : باید 


In [ ]:
result_slang2 = predict_analogy(dataset_slang2, dataset_slang_response2)

2332


100%|██████████| 2332/2332 [00:29<00:00, 80.17it/s]


acc_1:  0.003430531732418525
acc_3:  0.00686106346483705
acc_5:  0.007718696397941681
acc_10:  0.017152658662092625


In [ ]:
visualize(result_slang2,dataset_slang2, dataset_slang_response2 )

255


5it [00:00, 10470.05it/s]

The test is : بارون معادل عامیانه باران است و نیس معادل عامیانه [MASK] است
The answer is : نیست
The predicted result is : 
['برف', 'باران', 'اب', 'باد', 'تگرگ', 'خاک', 'اتش', 'هوا', 'ان', 'بارش']


The test is : بارون معادل عامیانه باران است و هندونه معادل عامیانه [MASK] است
The answer is : هندوانه
The predicted result is : 
['برف', 'باران', 'اب', 'باد', 'ان', 'تگرگ', 'بارش', 'خاک', 'رطوبت', 'بهار']


The test is : بارون معادل عامیانه باران است و شیطون معادل عامیانه [MASK] است
The answer is : شیطان
The predicted result is : 
['برف', 'تگرگ', 'باد', 'اب', 'باران', 'نان', 'سبزه', 'گندم', 'خاک', 'برنج']


The test is : بارون معادل عامیانه باران است و مطمعن معادل عامیانه [MASK] است
The answer is : مطمئن
The predicted result is : 
['برف', 'باران', 'تگرگ', 'باد', 'اب', 'ان', 'خاک', 'بارش', 'گندم', 'هوا']


The test is : بارون معادل عامیانه باران است و پولمون معادل عامیانه [MASK] است
The answer is : پولمان
The predicted result is : 
['برف', 'باد', 'اب', 'باران', 'تگرگ', 'نان', 'رطوبت', 'شراب',

In [ ]:
save_result(result_slang2,dataset_slang2, dataset_slang_response2, "slang_2.csv")

2332it [00:00, 965845.46it/s]

file saved to /content/drive/MyDrive/Analogy_NLP_Files/slang_2.csv


In [ ]:
df_result = pd.read_csv(ROOT+"slang_2.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df_slang,"cat_slang_2.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['semantic-slang']


2332it [00:00, 10434.84it/s]



 category is :  semantic-slang
Sample test : هیشکی معادل عامیانه هیچکس است و باس معادل عامیانه [MASK] است
acc_1:  0.003430531732418525
acc_3:  0.00686106346483705
acc_5:  0.007718696397941681
acc_10:  0.017152658662092625
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_slang_2.csv


In [ ]:
dataset_slang3, dataset_slang_response3 = generate_pattern(df_slang, pattern[3])

Pattern is : ['همانطور', 'که', 'برای', 'است', '[MASK]', 'برای', 'است']
Sample test : همانطور که هیچکس برای هیشکی است [MASK] برای باس است 
Sample test answer is : باید 


In [ ]:
result_slang3 = predict_analogy(dataset_slang3, dataset_slang_response3)

2332


100%|██████████| 2332/2332 [00:28<00:00, 80.59it/s]


acc_1:  0.0008576329331046312
acc_3:  0.002144082332761578
acc_5:  0.004288164665523156
acc_10:  0.007289879931389366


In [ ]:
visualize(result_slang3,dataset_slang3, dataset_slang_response3 )

2285


5it [00:00, 3347.41it/s]

The test is : همانطور که تکان برای تکون است [MASK] برای ترسوند است
The answer is : ترساند
The predicted result is : 
['و', 'تکان', '،', 'تکون', 'حرکت', 'اما', 'یا', 'نه', 'درد', 'اب']


The test is : همانطور که تکان برای تکون است [MASK] برای رسوند است
The answer is : رساند
The predicted result is : 
['تکون', 'و', '،', 'تکان', 'حرکت', 'اب', 'هم', 'یا', 'که', 'برش']


The test is : همانطور که تکان برای تکون است [MASK] برای ترکوند است
The answer is : ترکاند
The predicted result is : 
['تکون', 'و', 'حرکت', 'تکان', '،', 'درد', 'اب', 'یا', 'هم', 'برش']


The test is : همانطور که تکان برای تکون است [MASK] برای میشم است
The answer is : می‎شوم
The predicted result is : 
['تکون', 'تکان', 'و', '،', 'حرکت', 'اب', 'که', 'اما', 'درد', 'غذا']


The test is : همانطور که تکان برای تکون است [MASK] برای دشمنا است
The answer is : دشمن‎ها
The predicted result is : 
['تکون', 'و', 'تکان', '،', 'حرکت', 'هم', 'اب', 'درد', 'نه', 'غذا']




In [ ]:
save_result(result_slang3,dataset_slang3, dataset_slang_response3, "slang_3.csv")

2332it [00:00, 1446910.79it/s]

file saved to /content/drive/MyDrive/Analogy_NLP_Files/slang_3.csv


In [ ]:
df_result = pd.read_csv(ROOT+"slang_3.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df_slang,"cat_slang_3.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['semantic-slang']


2332it [00:00, 5892.84it/s]



 category is :  semantic-slang
Sample test : همانطور که هیچکس برای هیشکی است [MASK] برای باس است
acc_1:  0.0008576329331046312
acc_3:  0.002144082332761578
acc_5:  0.004288164665523156
acc_10:  0.007289879931389366
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_slang_3.csv


## with new method

In [ ]:
result_slang4, slang_tests1, slang_tests2, slang_responses4,total_relation_3, relation_3  = new_prediction_method(df_slang)

2332


2332it [01:46, 21.97it/s]


acc_1:  0.0
acc_3:  0.0
acc_5:  0.0004288164665523156
acc_10:  0.0004288164665523156


In [ ]:
save_result_new(result_slang4, slang_tests1, slang_tests2, slang_responses4, total_relation_3, relation_3, "new_method_slang.csv")

2332it [00:00, 909025.74it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/new_method_slang.csv


In [ ]:
df_slang_result4 = pd.read_csv(ROOT+"new_method_slang.csv", engine='python',encoding='utf-8', error_bad_lines=False)
df_slang_result4.head()
acurracy_category(df_slang_result4, df_slang,"cat_new_method_slang.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['semantic-slang']


2332it [00:00, 3428.06it/s]



 category is :  semantic-slang
Sample test1 : هیچکس [MASK] هیشکی است
Sample test2 : نمیداند باس [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0004288164665523156
acc_10:  0.0004288164665523156
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_new_method_slang.csv


In [ ]:
df_slang_result4 = pd.read_csv(ROOT+"new_method_slang.csv", engine='python',encoding='utf-8', error_bad_lines=False) 
df_slang_result4.head() 
relation_by_cat(df_slang_result4, df_slang, "slang_cat_rel.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['semantic-slang']


2332it [00:00, 4424.21it/s]


## Bokai analogy

In [ ]:
dataset_bokai, dataset_bokai_response = generate_pattern(df_bokai, pattern[0])

Pattern is : ['به', 'مانند', 'به', '[MASK]', 'است']
Sample test : سریع به سریعا مانند مخصوص به [MASK] است 
Sample test answer is : مخصوصا 


In [ ]:
result_bokai = predict_analogy(dataset_bokai, dataset_bokai_response)

31332


100%|██████████| 31332/31332 [06:31<00:00, 80.06it/s]


acc_1:  0.05323630792799693
acc_3:  0.12380314055917273
acc_5:  0.1674645729605515
acc_10:  0.23100983020554067


In [ ]:
visualize(result_bokai,dataset_bokai, dataset_bokai_response )

423


5it [00:00, 9668.75it/s]

The test is : متناظر به متناظرا مانند شجاع به [MASK] است
The answer is : شجاعانه
The predicted result is : 
['عقب', 'نظر', 'بالا', 'شجاع', 'قدرت', 'اینده', 'پایین', 'جلو', 'دریا', 'خیر']


The test is : متناظر به متناظرا مانند هوشمند به [MASK] است
The answer is : هوشمندانه
The predicted result is : 
['هوشمند', 'همراه', 'متصل', 'صورت', 'معنی', 'زیر', 'شکل', 'بالا', 'خودکار', 'پایین']


The test is : متناظر به متناظرا مانند فداکار به [MASK] است
The answer is : فداکارانه
The predicted result is : 
['خیر', 'خود', 'خدا', 'مرگ', 'ان', 'فرد', 'زمین', 'معنی', 'انسان', 'جان']


The test is : متناظر به متناظرا مانند عاقل به [MASK] است
The answer is : عاقلانه
The predicted result is : 
['عاقل', 'معقول', 'عالم', 'مجرد', 'بالغ', 'درستکار', 'بعد', 'فرد', 'سفیه', 'حقیقی']


The test is : متناظر به متناظرا مانند سیاستمدار به [MASK] است
The answer is : سیاستمدارانه
The predicted result is : 
['معنی', 'سیاستمدار', 'چپ', 'قدرت', 'وزیر', 'کابینه', 'دولت', 'راست', 'کشور', 'فرد']




In [ ]:
save_result(result_bokai,dataset_bokai, dataset_bokai_response, "bokai_1.csv")

31332it [00:00, 1278452.16it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/bokai_1.csv


In [ ]:
df_result = pd.read_csv(ROOT+"bokai_1.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df_bokai,"cat_bokai_1.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


[': 1st' ': 3rd' ': Antonym' ': Comparative' ': Country-Capital'
 ': Currency' ': Family' ': Infinitive-Past' ': Infinitive-Present'
 ': Nationality' ': Noun-Adverb' ': Province-Capital' ': Singular-Plural'
 ': Superlative' 'Adjective-Adverb:']


1260it [00:00, 3801.32it/s]



 category is :  : 1st
Sample test : رفتم به رفتیم مانند رقصیدم به [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


1332it [00:00, 10035.61it/s]


...


 category is :  : Singular-Plural
Sample test : کتاب به کتابها مانند زبان به [MASK] است
acc_1:  0.0
acc_3:  0.013333333333333334
acc_5:  0.03019607843137255
acc_10:  0.07450980392156863


1260it [00:00, 9842.76it/s]



 category is :  : Superlative
Sample test : سریع به سریعترین مانند بزرگ به [MASK] است
acc_1:  0.0
acc_3:  0.006349206349206349
acc_5:  0.01507936507936508
acc_10:  0.04206349206349207


1332it [00:00, 9857.02it/s] 



 category is :  Adjective-Adverb:
Sample test : سریع به سریعا مانند مخصوص به [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0007507507507507507
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_bokai_1.csv


In [ ]:
dataset_bokai2, dataset_bokai_response2 = generate_pattern(df_bokai, pattern[3])

Pattern is : ['همانطور', 'که', 'برای', 'است', '[MASK]', 'برای', 'است']
Sample test : همانطور که سریعا برای سریع است [MASK] برای مخصوص است 
Sample test answer is : مخصوصا 


In [ ]:
result_bokai2 = predict_analogy(dataset_bokai2, dataset_bokai_response2)

31332


100%|██████████| 31332/31332 [06:33<00:00, 79.71it/s]


acc_1:  0.046948806332184345
acc_3:  0.12456913060130219
acc_5:  0.19692327333078002
acc_10:  0.2817566704966169


In [ ]:
visualize(result_bokai2,dataset_bokai2, dataset_bokai_response2 )

6515


5it [00:00, 1666.26it/s]

The test is : همانطور که طرابلس برای لیبی است [MASK] برای صربستان است
The answer is : بلگراد
The predicted result is : 
['طرابلس', '،', 'و', 'رم', 'کوزوو', 'حلب', 'بلگراد', 'دمشق', 'استانبول', 'برلین']


The test is : همانطور که طرابلس برای لیبی است [MASK] برای بلغارستان است
The answer is : صوفیه
The predicted result is : 
['طرابلس', '،', 'و', 'حلب', 'رم', 'استانبول', 'دمشق', 'کوزوو', 'بیروت', 'ونیز']


The test is : همانطور که طرابلس برای لیبی است [MASK] برای بلژیک است
The answer is : بروکسل
The predicted result is : 
['طرابلس', '،', 'و', 'تونس', 'پاریس', 'دمشق', 'حلب', 'استانبول', 'الجزیره', 'ژنو']


The test is : همانطور که طرابلس برای لیبی است [MASK] برای ارمنستان است
The answer is : ایروان
The predicted result is : 
['،', 'طرابلس', 'و', 'دمشق', 'حلب', 'ایروان', 'بیروت', 'رم', 'سوریه', 'استانبول']


The test is : همانطور که طرابلس برای لیبی است [MASK] برای پاکستان است
The answer is : اسلام‌آباد
The predicted result is : 
['،', 'و', 'طرابلس', 'قدس', 'لاهور', 'حلب', 'الجزیره', 'دمشق'

In [ ]:
save_result(result_bokai2,dataset_bokai2, dataset_bokai_response2, "bokai_2.csv")

31332it [00:00, 1289276.30it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/bokai_2.csv


In [ ]:
df_result = pd.read_csv(ROOT+"bokai_2.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df_bokai,"cat_bokai_2.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


[': 1st' ': 3rd' ': Antonym' ': Comparative' ': Country-Capital'
 ': Currency' ': Family' ': Infinitive-Past' ': Infinitive-Present'
 ': Nationality' ': Noun-Adverb' ': Province-Capital' ': Singular-Plural'
 ': Superlative' 'Adjective-Adverb:']


1260it [00:00, 4756.53it/s]



 category is :  : 1st
Sample test : همانطور که رفتیم برای رفتم است [MASK] برای رقصیدم است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


1332it [00:00, 4670.63it/s]


...


 category is :  : Singular-Plural
Sample test : همانطور که کتابها برای کتاب است [MASK] برای زبان است
acc_1:  0.000392156862745098
acc_3:  0.00196078431372549
acc_5:  0.01019607843137255
acc_10:  0.02862745098039216


1260it [00:00, 9458.62it/s]



 category is :  : Superlative
Sample test : همانطور که سریعترین برای سریع است [MASK] برای بزرگ است
acc_1:  0.002380952380952381
acc_3:  0.013492063492063493
acc_5:  0.03968253968253968
acc_10:  0.10317460317460317


1332it [00:00, 9698.57it/s]



 category is :  Adjective-Adverb:
Sample test : همانطور که سریعا برای سریع است [MASK] برای مخصوص است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0007507507507507507
acc_10:  0.006006006006006006
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_bokai_2.csv


## with new mwthod

In [ ]:
analogy_bokai3,bokai_tests1, bokai_tests2, bokai_responses4,total_relation_4, relation_4  = new_prediction_method(df_bokai)

31332


31332it [24:50, 21.02it/s]


acc_1:  0.12492020937061152
acc_3:  0.14381463040980466
acc_5:  0.15479382101366015
acc_10:  0.18185880250223413


In [ ]:
save_result_new(analogy_bokai3,bokai_tests1, bokai_tests2, bokai_responses4,total_relation_4, relation_4, "new_method_bokai.csv")

31332it [00:00, 853339.13it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/new_method_bokai.csv


In [ ]:
df_bokai_result3 = pd.read_csv(ROOT+"new_method_bokai.csv", engine='python',encoding='utf-8', error_bad_lines=False)
df_bokai_result3.head()
acurracy_category(df_bokai_result3, df_bokai,"cat_new_method_bokai.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


[': 1st' ': 3rd' ': Antonym' ': Comparative' ': Country-Capital'
 ': Currency' ': Family' ': Infinitive-Past' ': Infinitive-Present'
 ': Nationality' ': Noun-Adverb' ': Province-Capital' ': Singular-Plural'
 ': Superlative' 'Adjective-Adverb:']


1260it [00:00, 11205.47it/s]



 category is :  : 1st
Sample test1 : رفتیم [MASK] رفتم است
Sample test2 : و رقصیدم [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


1332it [00:00, 11144.69it/s]


...


 category is :  : Singular-Plural
Sample test1 : کتابها [MASK] کتاب است
Sample test2 : صدها زبان [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.001568627450980392


1260it [00:00, 9645.91it/s] 



 category is :  : Superlative
Sample test1 : سریعترین [MASK] سریع است
Sample test2 : قطار بزرگ [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


1332it [00:00, 10238.35it/s]


 category is :  Adjective-Adverb:
Sample test1 : سریعا [MASK] سریع است
Sample test2 : خیلی مخصوص [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_new_method_bokai.csv


In [37]:
sem = [': Country-Capital',': Currency' ,': Family']
df_bokai_result = pd.read_csv(ROOT+"cat_new_method_bokai.csv", engine='python',encoding='utf-8', error_bad_lines=False) 
semantic_acc(df_bokai_result, df_bokai, sem)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


semantic results
length:  7004
acc_1:  0.5114220445459737
acc_3:  0.5608223872073101
acc_5:  0.5725299828669331
acc_10:  0.6046544831524843
syntactic results
length:  24328
acc_1:  0.01364682670174282
acc_3:  0.023758632028937837
acc_5:  0.03452811575139753
acc_10:  0.09688424860243336


In [ ]:
df_bokai_result4 = pd.read_csv(ROOT+"new_method_bokai.csv", engine='python',encoding='utf-8', error_bad_lines=False) 
df_bokai_result4.head() 
relation_by_cat(df_bokai_result4, df_bokai, "bokai_cat_rel.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


[': 1st' ': 3rd' ': Antonym' ': Comparative' ': Country-Capital'
 ': Currency' ': Family' ': Infinitive-Past' ': Infinitive-Present'
 ': Nationality' ': Noun-Adverb' ': Province-Capital' ': Singular-Plural'
 ': Superlative' 'Adjective-Adverb:']


1260it [00:00, 4744.94it/s]
1332it [00:00, 10154.39it/s]
1260it [00:00, 11405.95it/s]
2520it [00:00, 10784.13it/s]
5402it [00:00, 10145.03it/s]
1260it [00:00, 9515.33it/s] 
342it [00:00, 9626.10it/s]
1260it [00:00, 10046.12it/s]
1260it [00:00, 10710.25it/s]
1406it [00:00, 10689.17it/s]
1056it [00:00, 10367.63it/s]
7832it [00:00, 10515.52it/s]
2550it [00:00, 10340.98it/s]
1260it [00:00, 10255.66it/s]
1332it [00:00, 9807.12it/s]


## SAT analogy

In [ ]:
dataset_sat, dataset_sat_response = generate_pattern(df_sat_test, pattern[0])

Pattern is : ['به', 'مانند', 'به', '[MASK]', 'است']
Sample test : فقیر به بیپولی مانند ملتهب به [MASK] است 
Sample test answer is : دعوا 


In [ ]:
result_sat = predict_analogy(dataset_sat, dataset_sat_response)

1116


100%|██████████| 1116/1116 [00:13<00:00, 80.62it/s]


acc_1:  0.04659498207885305
acc_3:  0.12275985663082438
acc_5:  0.16577060931899643
acc_10:  0.2222222222222222


In [ ]:
visualize(result_sat,dataset_sat, dataset_sat_response)

490


5it [00:00, 7592.87it/s]

The test is : کوه به اورست مانند کشور به [MASK] است
The answer is : ایران
The predicted result is : 
['اورست', 'کوه', 'زمین', 'قله', 'دور', 'هند', 'بالا', 'ارتفاع', 'نپال', 'نام']


The test is : کوه به اورست مانند رودخانه به [MASK] است
The answer is : اروند
The predicted result is : 
['کوه', 'پایین', 'دریا', 'دره', 'اورست', 'رودخانه', 'بالا', 'زمین', 'اب', 'قله']


The test is : کوه به اورست مانند استان به [MASK] است
The answer is : خوزستان
The predicted result is : 
['اورست', 'کوه', 'چین', 'تبت', 'کوهستان', 'قله', 'زمین', 'ان', 'هند', 'ماه']


The test is : کوه به اورست مانند شهر به [MASK] است
The answer is : تهران
The predicted result is : 
['کوه', 'اورست', 'قله', 'شهر', 'دریا', 'کوهستان', 'بالا', 'پایین', 'صخره', 'جا']


The test is : کوه به اورست مانند دانشگاه به [MASK] است
The answer is : شریف
The predicted result is : 
['اورست', 'کوه', 'بالا', 'دور', 'زمین', 'پایین', 'قله', 'دانشگاه', 'ماه', 'کوهستان']




In [ ]:
save_result(result_sat,dataset_sat, dataset_sat_response, "sat_1.csv")

1116it [00:00, 441588.99it/s]

file saved to /content/drive/MyDrive/Analogy_NLP_Files/sat_1.csv


In [ ]:
df_result = pd.read_csv(ROOT+"sat_1.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df_sat_test,"cat_sat_1.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['Attribute  Agent Attribute:State (adjective:noun)  contentious:quarrels '
 'Attribute  Agent/Object Attribute:Typical Action (adjective:verb)  mutable:change '
 'Attribute  Item:Attribute (noun:adjective)  soldier:wounded '
 'Attribute  Object Attribute:Condition (adjective:adjective)  brittle:broken '
 'Attribute  Object:Typical Action (noun:verb)  glass:break '
 'CASE RELATIONS  Action:Object  tie:knot '
 'CASE RELATIONS  Action:Recipient  teach:student '
 'CASE RELATIONS  Agent:Object - Associated Objectplumber:pipe'
 'CASE RELATIONS  Agent:Object - Product tailor:suit '
 'CASE RELATIONS  Agent:Object - Raw Materialbaker:flour'
 'CASE RELATIONS  Agent:Recipient  doctor:patient '
 'CASE RELATIONS  Object:Instrument  violin:bow '
 'CASE RELATIONS  Object:Recipient  speech:audience '
 'CAUSE-PURPOSE  Cause:Compensatory Action  hunger:eat '
 'CAUSE-PURPOSE  Cause:Effect  joke:laughter '
 'CAUSE-PURPOSE  Instrument:Intended Action  gun:shoot '
 'CAUSE-PURPOSE Action/Activity:Goal' 'CAU

6it [00:00, 2556.20it/s]



 category is :  Attribute  Agent Attribute:State (adjective:noun)  contentious:quarrels 
Sample test : فقیر به بیپولی مانند ملتهب به [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


6it [00:00, 3460.65it/s]


...


 category is :  Space-Time Sequencecoda:symphony
Sample test : مقدمه به سخنرانی مانند پیشدرآمد به [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.16666666666666666
acc_10:  0.16666666666666666


55it [00:00, 2654.35it/s]



 category is :  Space-Time Time: Action/Activitysummer:harvest
Sample test : تابستان به درو مانند زمستان به [MASK] است
acc_1:  0.01818181818181818
acc_3:  0.07272727272727272
acc_5:  0.14545454545454545
acc_10:  0.21818181818181817


1it [00:00, 1499.57it/s]



 category is :  Space-Time Time:Associated Itemretirement:pension
Sample test : بازنشستگی به مستمری مانند نوزادی به [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_sat_1.csv


In [ ]:
dataset_sat2, dataset_sat_response2 = generate_pattern(df_sat_test, pattern[3])

Pattern is : ['همانطور', 'که', 'برای', 'است', '[MASK]', 'برای', 'است']
Sample test : همانطور که بیپولی برای فقیر است [MASK] برای ملتهب است 
Sample test answer is : دعوا 


In [ ]:
result_sat2 = predict_analogy(dataset_sat2, dataset_sat_response2)

1116


100%|██████████| 1116/1116 [00:13<00:00, 81.12it/s]


acc_1:  0.007168458781362007
acc_3:  0.02060931899641577
acc_5:  0.05465949820788531
acc_10:  0.11200716845878136


In [ ]:
visualize(result_sat2,dataset_sat2, dataset_sat_response2)

177


5it [00:00, 3575.10it/s]

The test is : همانطور که گچ برای گچکار است [MASK] برای بنا است
The answer is : آجر
The predicted result is : 
['گچ', 'اجر', '،', 'اب', 'و', 'سنگ', 'هم', 'خاک', 'سیمان', 'هوا']


The test is : همانطور که گچ برای گچکار است [MASK] برای خیاط است
The answer is : سوزن
The predicted result is : 
['گچ', '،', 'و', 'چوب', 'یا', 'هم', 'چرم', 'پارچه', 'لباس', 'مو']


The test is : همانطور که گچ برای گچکار است [MASK] برای آشپز است
The answer is : ماهیتابه
The predicted result is : 
['گچ', '،', 'و', 'اب', 'غذا', 'اجر', 'چوب', 'خاک', 'یا', 'هم']


The test is : همانطور که گچ برای گچکار است [MASK] برای راننده است
The answer is : ماشین
The predicted result is : 
['گچ', '،', 'و', 'سیمان', 'چوب', 'اب', 'اجر', 'هم', 'سنگ', 'بتن']


The test is : همانطور که گچ برای گچکار است [MASK] برای دانشجو است
The answer is : کتاب
The predicted result is : 
['گچ', '،', 'و', 'یا', 'هنر', 'اب', 'هم', 'سیمان', 'اما', 'اجر']




In [ ]:
save_result(result_sat2,dataset_sat2, dataset_sat_response2, "sat_2.csv")

1116it [00:00, 672052.16it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/sat_2.csv


In [ ]:
df_result = pd.read_csv(ROOT+"sat_2.csv", engine='python',encoding='utf-8', error_bad_lines=False)
acurracy_category(df_result, df_sat_test,"cat_sat_2.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['Attribute  Agent Attribute:State (adjective:noun)  contentious:quarrels '
 'Attribute  Agent/Object Attribute:Typical Action (adjective:verb)  mutable:change '
 'Attribute  Item:Attribute (noun:adjective)  soldier:wounded '
 'Attribute  Object Attribute:Condition (adjective:adjective)  brittle:broken '
 'Attribute  Object:Typical Action (noun:verb)  glass:break '
 'CASE RELATIONS  Action:Object  tie:knot '
 'CASE RELATIONS  Action:Recipient  teach:student '
 'CASE RELATIONS  Agent:Object - Associated Objectplumber:pipe'
 'CASE RELATIONS  Agent:Object - Product tailor:suit '
 'CASE RELATIONS  Agent:Object - Raw Materialbaker:flour'
 'CASE RELATIONS  Agent:Recipient  doctor:patient '
 'CASE RELATIONS  Object:Instrument  violin:bow '
 'CASE RELATIONS  Object:Recipient  speech:audience '
 'CAUSE-PURPOSE  Cause:Compensatory Action  hunger:eat '
 'CAUSE-PURPOSE  Cause:Effect  joke:laughter '
 'CAUSE-PURPOSE  Instrument:Intended Action  gun:shoot '
 'CAUSE-PURPOSE Action/Activity:Goal' 'CAU

6it [00:00, 1284.17it/s]



 category is :  Attribute  Agent Attribute:State (adjective:noun)  contentious:quarrels 
Sample test : همانطور که بیپولی برای فقیر است [MASK] برای ملتهب است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


6it [00:00, 3494.28it/s]


...


 category is :  Space-Time Sequencecoda:symphony
Sample test : همانطور که سخنرانی برای مقدمه است [MASK] برای پیشدرآمد است
acc_1:  0.0
acc_3:  0.16666666666666666
acc_5:  0.16666666666666666
acc_10:  0.16666666666666666


55it [00:00, 3411.16it/s]



 category is :  Space-Time Time: Action/Activitysummer:harvest
Sample test : همانطور که درو برای تابستان است [MASK] برای زمستان است
acc_1:  0.01818181818181818
acc_3:  0.01818181818181818
acc_5:  0.07272727272727272
acc_10:  0.10909090909090909


1it [00:00, 1731.04it/s]



 category is :  Space-Time Time:Associated Itemretirement:pension
Sample test : همانطور که مستمری برای بازنشستگی است [MASK] برای نوزادی است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_sat_2.csv


## with new method

In [ ]:
analogy_sat3,sat_tests1, sat_tests2, sat_responses4,total_relation_5, relation_5  = new_prediction_method(df_sat_test)

1116


1116it [00:52, 21.07it/s]


acc_1:  0.010752688172043012
acc_3:  0.021505376344086023
acc_5:  0.034050179211469536
acc_10:  0.06003584229390681


In [ ]:
save_result_new(analogy_sat3,sat_tests1, sat_tests2, sat_responses4,total_relation_5, relation_5, "new_method_sat.csv")

1116it [00:00, 556448.32it/s]


file saved to /content/drive/MyDrive/Analogy_NLP_Files/new_method_sat.csv


In [ ]:
df_sat_result3 = pd.read_csv(ROOT+"new_method_sat.csv", engine='python',encoding='utf-8', error_bad_lines=False)
df_sat_result3.head()
df_sat = df_sat.rename(columns={'Desc': 'cat'})
acurracy_category(df_sat_result3, df_sat,"cat_new_method_sat.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['Attribute  Agent Attribute:State (adjective:noun)  contentious:quarrels '
 'Attribute  Agent/Object Attribute:Typical Action (adjective:verb)  mutable:change '
 'Attribute  Item:Attribute (noun:adjective)  soldier:wounded '
 'Attribute  Object Attribute:Condition (adjective:adjective)  brittle:broken '
 'Attribute  Object:Typical Action (noun:verb)  glass:break '
 'CASE RELATIONS  Action:Object  tie:knot '
 'CASE RELATIONS  Action:Recipient  teach:student '
 'CASE RELATIONS  Agent:Object - Associated Objectplumber:pipe'
 'CASE RELATIONS  Agent:Object - Product tailor:suit '
 'CASE RELATIONS  Agent:Object - Raw Materialbaker:flour'
 'CASE RELATIONS  Agent:Recipient  doctor:patient '
 'CASE RELATIONS  Object:Instrument  violin:bow '
 'CASE RELATIONS  Object:Recipient  speech:audience '
 'CAUSE-PURPOSE  Cause:Compensatory Action  hunger:eat '
 'CAUSE-PURPOSE  Cause:Effect  joke:laughter '
 'CAUSE-PURPOSE  Instrument:Intended Action  gun:shoot '
 'CAUSE-PURPOSE Action/Activity:Goal' 'CAU

4it [00:00, 2105.57it/s]



 category is :  Attribute  Agent Attribute:State (adjective:noun)  contentious:quarrels 
Sample test1 : گچ [MASK] معلم است
Sample test2 : هم نجار [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


4it [00:00, 4044.65it/s]


...


 category is :  Space-Time Sequencecoda:symphony
Sample test1 : زخمه [MASK] تار است
Sample test2 : دوم میخ [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0


11it [00:00, 4039.34it/s]



 category is :  Space-Time Time: Action/Activitysummer:harvest
Sample test1 : دانشاموز [MASK] معلم است
Sample test2 : دوم زندانبان [MASK] است
acc_1:  0.0
acc_3:  0.09090909090909091
acc_5:  0.09090909090909091
acc_10:  0.2727272727272727


2it [00:00, 2243.54it/s]



 category is :  Space-Time Time:Associated Itemretirement:pension
Sample test1 : زخمه [MASK] تار است
Sample test2 : دوم سیم [MASK] است
acc_1:  0.0
acc_3:  0.0
acc_5:  0.0
acc_10:  0.0
file saved to /content/drive/MyDrive/Analogy_NLP_Files/cat_new_method_sat.csv


In [ ]:
df_set_result4 = pd.read_csv(ROOT+"new_method_sat.csv", engine='python',encoding='utf-8', error_bad_lines=False) 
df_set_result4.head() 
relation_by_cat(df_set_result4, df_sat_test, "set_cat_rel.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


['Attribute  Agent Attribute:State (adjective:noun)  contentious:quarrels '
 'Attribute  Agent/Object Attribute:Typical Action (adjective:verb)  mutable:change '
 'Attribute  Item:Attribute (noun:adjective)  soldier:wounded '
 'Attribute  Object Attribute:Condition (adjective:adjective)  brittle:broken '
 'Attribute  Object:Typical Action (noun:verb)  glass:break '
 'CASE RELATIONS  Action:Object  tie:knot '
 'CASE RELATIONS  Action:Recipient  teach:student '
 'CASE RELATIONS  Agent:Object - Associated Objectplumber:pipe'
 'CASE RELATIONS  Agent:Object - Product tailor:suit '
 'CASE RELATIONS  Agent:Object - Raw Materialbaker:flour'
 'CASE RELATIONS  Agent:Recipient  doctor:patient '
 'CASE RELATIONS  Object:Instrument  violin:bow '
 'CASE RELATIONS  Object:Recipient  speech:audience '
 'CAUSE-PURPOSE  Cause:Compensatory Action  hunger:eat '
 'CAUSE-PURPOSE  Cause:Effect  joke:laughter '
 'CAUSE-PURPOSE  Instrument:Intended Action  gun:shoot '
 'CAUSE-PURPOSE Action/Activity:Goal' 'CAU

6it [00:00, 2388.56it/s]
6it [00:00, 3400.79it/s]
28it [00:00, 4322.59it/s]
6it [00:00, 3567.09it/s]
55it [00:00, 3376.86it/s]
45it [00:00, 2415.70it/s]
10it [00:00, 4149.08it/s]
66it [00:00, 2196.98it/s]
78it [00:00, 3812.38it/s]
28it [00:00, 2622.44it/s]
21it [00:00, 2984.87it/s]
15it [00:00, 2847.20it/s]
6it [00:00, 3320.91it/s]
10it [00:00, 2788.58it/s]
10it [00:00, 3077.26it/s]
10it [00:00, 3738.57it/s]
21it [00:00, 3820.61it/s]
28it [00:00, 1295.78it/s]
21it [00:00, 3762.35it/s]
10it [00:00, 3269.39it/s]
10it [00:00, 3480.75it/s]
15it [00:00, 3676.20it/s]
6it [00:00, 3665.28it/s]
1it [00:00, 1315.24it/s]
21it [00:00, 2048.33it/s]
10it [00:00, 3696.72it/s]
15it [00:00, 3605.42it/s]
10it [00:00, 3672.45it/s]
28it [00:00, 3982.38it/s]
36it [00:00, 1994.20it/s]
6it [00:00, 3501.09it/s]
6it [00:00, 3601.29it/s]
6it [00:00, 1412.86it/s]
21it [00:00, 4235.86it/s]
15it [00:00, 3570.63it/s]
10it [00:00, 5675.65it/s]
6it [00:00, 2146.71it/s]
6it [00:00, 1734.38it/s]
15it [00:00, 2812.45it/

# GPT2

In [ ]:
normalizer = hazm.Normalizer(persian_numbers=False)

def normalize_input(text):
    text = normalizer.normalize(text)
    return text

def sents_as_output(text, num_sents=1):
    sents = hazm.sent_tokenize(text)
    if num_sents > 0:
        return " ".join(sents[:num_sents])
    return " ".join(sents[0])


generator = pipeline('text-generation', "HooshvareLab/gpt2-fa")

Downloading:   0%|          | 0.00/808 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/463M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/728 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/855k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/104 [00:00<?, ?B/s]

In [ ]:
text = "تهران به ایران مانند پاریس به"
text = normalize_input(text)

outputs = generator(text)
for output in outputs:
    generated = output["generated_text"]
    print(sents_as_output(generated))

Setting `pad_token_id` to `eos_token_id`:5 for open-end generation.


تهران به ایران مانند پاریس به چشم می‌خورد.
